In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load training and test data
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')

In [ ]:
# balanced dataset
%run scripts/helper.py
train = balanced_sample(train)

In [ ]:
# response variable
y = train.solved_status

In [ ]:
# preprocess skills feature

# determine unique skills
skills = set()
for m in train.skills:
    skills.update(g for g in m.split('|'))
skills = sorted(skills)

#make a column for each skill
for skill in skills:
    train[skill] = [int(skill in s.split('|')) for s in train.skills]

In [ ]:
# do same calculation for test set as well
for skill in skills:
    test[skill] = [int(skill in s.split('|')) for s in test.skills]

In [ ]:
# determine problem tags
problem_tags = set()
for tag1, tag2, tag3, tag4, tag5 in zip(train.tag1, train.tag2, train.tag3, train.tag4, train.tag5):
    problem_tags.update([tag1, tag2, tag3, tag4, tag5])
problem_tags = sorted(problem_tags)

In [ ]:
problem_tags[0] = 'Not Specified'

In [ ]:
# make a column for each problem type
for problem_tag in problem_tags:
    train[problem_tag] = [int(problem_tag in tags) for tags in train[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

In [ ]:
for problem_tag in problem_tags:
    test[problem_tag] = [int(problem_tag in tags) for tags in test[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

## Prepare a very simple model based on just the accuracy of the problem

In [ ]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [ ]:
# model = build_extreme_gradient_boosting(train, test)
# model = build_random_forest_classifier(train, test)
# model = build_logistic_regression_model(train, test)
model = build_knn_classifier(train, test)
# model = build_extra_trees_classifier(train, test)

In [ ]:
scores = eval_models([model], train, y)

In [ ]:
print 'Mean: %f and Standard Deviation: %f' %(np.mean(scores), np.std(scores))

In [ ]:
predsTrain, predsTest, y_train, y_test = analyze_output(train)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, predsTest)

In [ ]:
misclassified = train.ix[(y_test != predsTest).index.values]
misclassified['preds'] = predsTest

In [ ]:
misclassified_class_0 = misclassified[misclassified.solved_status == 0]

In [ ]:
misclassified_class_0[['accuracy', 'solved_status', 'preds']]

In [ ]:
sns.pointplot(x="solved_status", y="accuracy", data=misclassified);

In [ ]:
# train model
import time

start_time = time.time()

model1.fit(train, y)
model2.fit(train, y)
model3.fit(train, y)

elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

In [ ]:
predictions1  = model1.predict(test)
predictions2 = model2.predict(test)
predictions3 = model3.predict(test)

ensemble_preds = np.array(majority_voting([predictions1, predictions2, predictions3]))
# predictions = model.predict(test)

In [ ]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': ensemble_preds.astype(int)})
submission_df.to_csv('submissions/thirty_second_ensemble_xgbs.csv', index=False)